# Capstone Project Notebook (part 1 of 3)
### by Dimitar Kunev
This notebook will be used as the primary vehicle for my capstone project.

In [1]:
# set up the libraries
#    this list will grow as I progress through the project
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
# from bs4 import BeautifulSoup

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
# get the table from the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
source = lh.fromstring(page.content)
tr_els = source.xpath('//tr')
headers=[]
i=0

for t in tr_els[0]:
    i+=1
    name=t.text_content()
    headers.append((name,[]))

for j in range(1, len(tr_els)):
    T=tr_els[j]
    if len(T) != 3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content()
        headers[i][1].append(data)
        i+=1

Dict={title:column for (title,column) in headers}
df=pd.DataFrame(Dict)

In [3]:
len(df.index)

288

In [3]:
df1 = pd.DataFrame(df[df.Borough != 'Not assigned'])
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [88]:
len(df1.index)

211

In [4]:
# get rid of new line characters in the dataframe and column heading:
df1 = df1.replace('\n', '', regex=True)
df1.columns = df1.columns.str.strip()
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
# sort the data frame by Postcode... just to be on the safe side
df1.sort_values(by=['Postcode'], inplace=True)
df1.reset_index(inplace=True, drop=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek


### Assumption:
There are no postal codes with more than 1 neighborhood where at least one of the neighborhoods is listed as "Not assigned" ***and*** at least one other is known.

In [6]:
# merge the neighbourhoods by postal code and borough
df1 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# copy borough name to neighborhood when latter = "Not assigned"
for i in range(0,len(df1.index)):
    if df1.iloc[i,2] == 'Not assigned':
        df1.iloc[i,2] = df1.iloc[i,1]

In [8]:
df1.shape

(103, 3)

In [9]:
df1.to_csv('toronto.csv')